In [1]:
!pip uninstall rulprediction --yes

Found existing installation: rulprediction 1.0
Uninstalling rulprediction-1.0:
  Successfully uninstalled rulprediction-1.0


In [2]:
!pip install git+https://github.com/sashaatlasov/RUL-Prediction.git

  Cloning https://github.com/sashaatlasov/RUL-Prediction.git to /private/var/folders/cr/12mnwryn4bj3dhrct6_xvw340000gn/T/pip-req-build-cciikz2i
  Running command git clone --filter=blob:none --quiet https://github.com/sashaatlasov/RUL-Prediction.git /private/var/folders/cr/12mnwryn4bj3dhrct6_xvw340000gn/T/pip-req-build-cciikz2i
  Resolved https://github.com/sashaatlasov/RUL-Prediction.git to commit 71e626f695321c94a68b7bbac515194cfe198789
  Preparing metadata (setup.py) ... done
  Created wheel for rulprediction: filename=rulprediction-1.0-py3-none-any.whl size=8881 sha256=f67369c7dd1349b09004a25720cd305eaac2a7faedf0d7e59c9c266a4c8071e9
  Stored in directory: /private/var/folders/cr/12mnwryn4bj3dhrct6_xvw340000gn/T/pip-ephem-wheel-cache-xu3glqsr/wheels/d9/26/2f/4a379e714991d395b358c8aebe5b708c26e61c28df8e5e8036
Successfully built rulprediction


In [3]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

In [4]:
class CMAPSS(Dataset):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        target = torch.tensor(self.y[idx], dtype=torch.float32)
        sensors = torch.tensor(self.X[idx], dtype=torch.float32)
        return target, sensors.squeeze(-1)

In [5]:
def get_data(k):
    X_train = sio.loadmat(f'data/F00{k}/trainX.mat')['trainX']
    X_test = sio.loadmat(f'data/F00{k}/testX.mat')['testX']

    y_train = sio.loadmat(f'data/F00{k}/trainY.mat')['trainY'].T
    y_test = sio.loadmat(f'data/F00{k}/testY.mat')['testY'].T
    
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.2, random_state=1)
    
    train_data = CMAPSS(X_train, y_train) 
    val_data = CMAPSS(X_val, y_val) 
    test_data = CMAPSS(X_test, y_test) 
                        
    train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=256, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=256)
    test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=256)
                        
    return train_loader, val_loader, test_loader

In [6]:
from rulpred.RULprediction import RULpredictor
from rulpred.training import *

from hparams import config

def main(k, device='cpu', num_epochs=100):
    if k % 2 == 0:
        win_size = 60
    else:
        win_size = 40
        
    model = RULpredictor(win_size)
    model.to(device)

    optim = torch.optim.Adam(model.parameters(), lr=config["learning_rate_start"])
    train_dataloader, val_dataloader, test_dataloader = get_data(k)

    for i in range(1, config["epochs"] + 1):
        loss = train_epoch(model, train_dataloader, optim, device=device)
        if i % 5 == 0:
            val_loss = validate(model, val_dataloader, device)
    test_loss = validate(model, test_dataloader, device)
    #torch.save(model, f'checkpoints/Model_F00{k}.pth')

In [7]:
main(1)

NameError: name 'dec_seq_len' is not defined